In [1]:
import datefinder
import numpy as np
import easyocr, torch
import language_tool_python
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline 

ModuleNotFoundError: No module named 'datefinder'

In [5]:
tool = language_tool_python.LanguageTool('en-US')
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
classifier = pipeline(
                    "zero-shot-classification",
                    model="facebook/bart-large-mnli",
                    device = 0 if torch.cuda.is_available() else -1
                    )

candidates = [
            'Topic selection & registration',
            'Create & submit Topic assesment form',
            'Create & submit Charter form',
            'Create & submit proposal draft',
            'Proposal Presenation',
            'Identify process deliverables',
            'Select technologies',
            'Collecting the data set',
            'Training relevant data sets',
            'Design wireframes',
            'Database design',
            'Backend development',
            'Frontend development',
            'Unit test',
            'Integration test' ,
            'QA test',
            'Working on exsisting bugs',
            'Fix bugs',
            'Finalize the system',
            'Final report creation & submission',
            'Final Presentation',
            'Releasing Phase',
            ]

In [20]:
def text_extraction(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path, detail = 0)
    result = ' '.join(result)
    matches = tool.check(result)
    corrected = language_tool_python.utils.correct(result, matches)
    return corrected

def zero_shot_text_classification(corrected_text):
    try:
        tokenized_products = tokenizer.encode_plus(
                                                    corrected_text, 
                                                    return_tensors='pt', 
                                                    max_length=len(corrected_text.split()), 
                                                    pad_to_max_length=True
                                                    )
        preditions = nli_model(**tokenized_products)
        logits = preditions.logits
        probs = torch.softmax(logits, dim=-1)
        probs = probs.detach().numpy()
        probs = np.array(probs, dtype=np.object)
        probs = probs[:, 1]
        output = candidates[np.argmax(probs)]
    except:
        predition = classifier(corrected_text, candidates, multi_label=True)
        output = predition['labels'][np.argmax(predition['scores'])]
    return output

def extract_date(text):
    matches = datefinder.find_dates(text)
    for match in matches:
        print(match)

def component_01_pipeline(image_path):
    corrected_text = text_extraction(image_path)
    zero_shot_text_classification(corrected_text)
    print("This Ducemnt is about: ", zero_shot_text_classification(corrected_text))
    # extract_date(corrected_text)

In [21]:
image_path = 'data/ocr/HandWrittenData/1.jpg'
component_01_pipeline(image_path)

c:\Users\Legion\.conda\envs\torch113\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
C:\Users\Legion\AppData\Local\Temp\ipykernel_7028\629145146.py:21: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  probs = np.array(probs, dtype=np.object)


This Ducemnt is about:  Topic selection & registration
